In [1]:

import numpy as np
import pandas as pd
import requests
import json
import time
import env

In [16]:
sql_url = env.get_url('leavitt_1861')

# Get all strava activities for my account

In [17]:

## Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
## If access_token has expired then use the refresh_token to get the new access_token
if strava_tokens['expires_at'] < time.time():
#Make Strava auth API call with current refresh token
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': 96678,
                                'client_secret': '10705c973422a20357c7b7f89d21fe8189c90ad3',
                                'grant_type': 'refresh_token',
                                'refresh_token': strava_tokens['b4faf19c870a011ea3ead0d7a08af002d51b4ec8']
                                }
                    )
#Save response as json in new variable
    new_strava_tokens = response.json()
# Save new tokens to file
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)
#Use new Strava tokens from now
    strava_tokens = new_strava_tokens

## Create the dataframe ready for the API call to store your activity data
columns = [
       'id', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'has_heartrate', 'average_heartrate',
       'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option',
       'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'pr_count', 'total_photo_count', 'has_kudoed',
       'suffer_score'
    ]



activities = pd.DataFrame(
    columns = columns
)

#Loop through all activities
page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']

while True:
    
    # get page of activities from Strava
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()
# if no results then exit loop
    if (not r):
        break
    
    # otherwise add new data to dataframe
    for x in range(len(r)):
        for col in columns:
            activities.loc[x + (page-1)*200,col] = r[x].get(col, None)  


# increment page
    page += 1


,id,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,start_date,...,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,pr_count,total_photo_count,has_kudoed,suffer_score
0,8072394788,Long Run,32447.9,9596,10215,157.0,Run,Run,0,2022-11-05T11:49:42Z,...,True,340.8,242.6,8640585858,8640585858,garmin_ping_245898950993,0,0,False,277.0
1,8066883741,Morning Run,7149.8,2210,2439,29.0,Run,Run,None,2022-11-04T12:09:12Z,...,True,253.4,230.4,8634347762,8634347762,garmin_ping_245757233342,0,0,False,33.0
2,8062278455,Morning Run,16199.4,4674,5148,56.0,Run,Run,0,2022-11-03T11:16:34Z,...,True,252.6,226.4,8629166652,8629166652,garmin_ping_245626723834,0,0,False,106.0
3,8057324394,Morning Run,8826.4,2746,2799,38.0,Run,Run,None,2022-11-02T11:49:29Z,...,True,252.8,232.4,8623587866,8623587866,garmin_ping_245494378642,0,0,False,43.0
4,8052346673,Morning Run,20539.5,6189,6301,111.0,Run,Run,0,2022-11-01T10:55:36Z,...,True,266.2,225.0,8617965934,8617965934,garmin_ping_245356059169,0,0,False,113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,1927729,"09/21/2011 Istres, Provence-Alpes-Côte d'Azur,...",3421.2,1186,1186,0.0,Run,Run,None,2011-09-21T04:36:16Z,...,False,40.4,27.0,2294403,2294403,2011-09-21T04:36:16Z,0,0,False,None
2060,1927731,"09/19/2011 Istres, Provence-Alpes-Côte d'Azur,...",3509.1,1231,1231,15.0,Run,Run,None,2011-09-19T04:35:45Z,...,False,43.8,28.0,2294408,2294408,2011-09-19T04:35:45Z,0,0,False,None
2061,1927730,"09/16/2011 Istres, Provence-Alpes-Côte d'Azur,...",3739.8,1387,1387,0.0,Run,Run,None,2011-09-16T04:39:17Z,...,False,39.4,29.0,2294407,2294407,2011-09-16T04:39:17Z,0,0,False,None
2062,1927732,"09/15/2011 Istres, Provence-Alpes-Côte d'Azur,...",5170.4,1706,1706,41.6,Run,Run,None,2011-09-15T04:34:15Z,...,False,47.2,27.6,2294409,2294409,2011-09-15T04:34:15Z,0,0,False,None


### Lat/Long is in the table as lists.  Separate into their own cols and drop original

In [242]:
start = pd.DataFrame(activities.start_latlng.to_list(), columns=['start_lat', 'start_long'])
end = pd.DataFrame(activities.end_latlng.to_list(), columns=['end_lat', 'end_long'])
activities = pd.concat([activities, start, end], axis=1)
activities.drop(columns=['start_latlng', 'end_latlng'], inplace=True)

### Set the activity ID as an int and make it the index

In [246]:
activities.id = activities.id.astype(int)
activities = activities.set_index('id')

### Send to SQL, replacing the table if it exists

In [247]:
activities.to_sql('activities', sql_url, if_exists='replace')

2064

# Next.  Get only new activities?

In [15]:

## Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
## If access_token has expired then use the refresh_token to get the new access_token
if strava_tokens['expires_at'] < time.time():
#Make Strava auth API call with current refresh token
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': 96678,
                                'client_secret': '10705c973422a20357c7b7f89d21fe8189c90ad3',
                                'grant_type': 'refresh_token',
                                'refresh_token': strava_tokens['b4faf19c870a011ea3ead0d7a08af002d51b4ec8']
                                }
                    )
#Save response as json in new variable
    new_strava_tokens = response.json()
# Save new tokens to file
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)
#Use new Strava tokens from now
    strava_tokens = new_strava_tokens
    
# Get ID of latest activity in database
latest = pd.read_sql('''SELECT MAX(id) as latest FROM activities''', 
            env.get_url(env.user, env.host, env.password, 'leavitt_1861')).latest[0]


columns = [
       'id', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'has_heartrate', 'average_heartrate',
       'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option',
       'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'pr_count', 'total_photo_count', 'has_kudoed',
       'suffer_score'
    ]


activities = pd.DataFrame(columns = columns)
# get one page of activities
r = requests.get(url + '?access_token=' + access_token)
r = r.json()

for x in range(10):
    if r[x]['id'] > latest:
        for col in columns:
            activities.loc[x,col] = r[x].get(col, None)  




OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '5vovOCuuQgqAOc3HnO4Y6J5vAUHRQCXT' ([Errno 8] nodename nor servname provided, or not known)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,start_date,start_date_local,...,upload_id_str,external_id,pr_count,total_photo_count,has_kudoed,suffer_score,start_lat,start_long,end_lat,end_long
id,,,,,,,,,,,,,,,,,,,,,
8080088087,Afternoon Run,0.4,0,3,0.0,Run,Run,None,2022-11-06T23:44:15Z,2022-11-06T17:44:15Z,...,8649460335,garmin_ping_246090533102,0,0,False,None,29.560139,-98.463326,29.560114,-98.463331


In [27]:
start = pd.DataFrame(activities.start_latlng.to_list(), columns=['start_lat', 'start_long'])
end = pd.DataFrame(activities.end_latlng.to_list(), columns=['end_lat', 'end_long'])
activities = pd.concat([activities, start, end], axis=1)
activities.drop(columns=['start_latlng', 'end_latlng'], inplace=True)

NameError: name 'activities' is not defined

In [280]:
activities.id = activities.id.astype(int)
activities = activities.set_index('id')

0

# modularize repetitive functions?

In [5]:

import numpy as np
import pandas as pd
import requests
import json
import time
import env

In [6]:
def get_strava_token():
        ## Get the tokens from file to connect to Strava
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)
    ## If access_token has expired then use the refresh_token to get the new access_token
    if strava_tokens['expires_at'] < time.time():
    #Make Strava auth API call with current refresh token
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': 96678,
                                    'client_secret': '10705c973422a20357c7b7f89d21fe8189c90ad3',
                                    'grant_type': 'b4faf19c870a011ea3ead0d7a08af002d51b4ec8',
                                    'refresh_token': strava_tokens['b4faf19c870a011ea3ead0d7a08af002d51b4ec8']
                                    }
                        )
    #Save response as json in new variable
        new_strava_tokens = response.json()
    # Save new tokens to file
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
    #Use new Strava tokens from now
        strava_tokens = new_strava_tokens
    return strava_tokens

In [7]:
def init_activities_df():
    columns = [
       'id', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'has_heartrate', 'average_heartrate',
       'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option',
       'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'pr_count', 'total_photo_count', 'has_kudoed',
       'suffer_score'
    ]


    activities = pd.DataFrame(columns = columns)
    return activities

In [8]:
def fix_lat_long(activities):
    start = pd.DataFrame(activities.start_latlng.to_list(), columns=['start_lat', 'start_long'])
    end = pd.DataFrame(activities.end_latlng.to_list(), columns=['end_lat', 'end_long'])
    activities = pd.concat([activities, start, end], axis=1)
    activities.drop(columns=['start_latlng', 'end_latlng'], inplace=True)
    return activities

In [9]:
def get_all_activities():
    strava_tokens = get_strava_token()
    activities = init_activities_df()
    columns = activities.columns
    
    #Loop through all activities
    page = 1
    url = "https://www.strava.com/api/v3/activities"
    access_token = strava_tokens['access_token']

    while True:

        # get page of activities from Strava
        r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
        r = r.json()
    # if no results then exit loop
        if (not r):
            break

        # otherwise add new data to dataframe
        for x in range(len(r)):
            for col in columns:
                activities.loc[x + (page-1)*200,col] = r[x].get(col, None)  


    # increment page
        page += 1
    # fix lat/long
    activities = fix_lat_long(activities)
    # set index
    activities.id = activities.id.astype(int)
    activities = activities.set_index('id')
    
    activities.to_sql('activities', env.get_url('leavitt_1861'), if_exists='replace')
    
    return activities

In [10]:
def get_new_activities(depth=10):
    strava_tokens = get_strava_token()
    activities = init_activities_df()
    columns = activities.columns

    # Get ID of latest activity in database
    latest = pd.read_sql('''SELECT MAX(id) as latest FROM activities''', 
            env.get_url('leavitt_1861')).latest[0]
    
    url = "https://www.strava.com/api/v3/activities"
    access_token = strava_tokens['access_token']
    
    # get one page of activities
    r = requests.get(url + '?access_token=' + access_token)
    r = r.json()

    # check last 10 
    for x in range(depth):
        if r[x]['id'] > latest:
            for col in columns:
                activities.loc[x,col] = r[x].get(col, None)  
                
    # fix lat/long
    activities = fix_lat_long(activities)
    # set index
    activities.id = activities.id.astype(int)
    activities = activities.set_index('id')
    
    activities.to_sql('activities', env.get_url('leavitt_1861'), if_exists='append')
    
    return activities

In [11]:
df = get_new_activities()

KeyError: 'b4faf19c870a011ea3ead0d7a08af002d51b4ec8'

In [14]:
df.shape

(2064, 48)

In [ ]:
acquire

In [1]:
import pandas as pd
import numpy as np
import acquire
import env

In [2]:
df = acquire.get_new_activities()

In [4]:
df

,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,start_date,start_date_local,...,upload_id_str,external_id,pr_count,total_photo_count,has_kudoed,suffer_score,start_lat,start_long,end_lat,end_long
id,,,,,,,,,,,,,,,,,,,,,
8072394788,Long Run,32447.9,9596,10215,157.0,Run,Run,0,2022-11-05T11:49:42Z,2022-11-05T06:49:42Z,...,8640585858,garmin_ping_245898950993,0,0,False,277.0,29.556511,-98.499868,29.556463,-98.499863
8066883741,Morning Run,7149.8,2210,2439,29.0,Run,Run,None,2022-11-04T12:09:12Z,2022-11-04T07:09:12Z,...,8634347762,garmin_ping_245757233342,0,0,False,33.0,29.559992,-98.463155,29.560667,-98.459519
8062278455,Morning Run,16199.4,4674,5148,56.0,Run,Run,0,2022-11-03T11:16:34Z,2022-11-03T06:16:34Z,...,8629166652,garmin_ping_245626723834,0,0,False,106.0,29.559993,-98.463173,29.560691,-98.459477
8057324394,Morning Run,8826.4,2746,2799,38.0,Run,Run,None,2022-11-02T11:49:29Z,2022-11-02T06:49:29Z,...,8623587866,garmin_ping_245494378642,0,0,False,43.0,29.559972,-98.463174,29.559423,-98.461281
8052346673,Morning Run,20539.5,6189,6301,111.0,Run,Run,0,2022-11-01T10:55:36Z,2022-11-01T05:55:36Z,...,8617965934,garmin_ping_245356059169,0,0,False,113.0,29.560015,-98.463088,29.560619,-98.462592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927729,"09/21/2011 Istres, Provence-Alpes-Côte d'Azur,...",3421.2,1186,1186,0.0,Run,Run,None,2011-09-21T04:36:16Z,2011-09-21T06:36:16Z,...,2294403,2011-09-21T04:36:16Z,0,0,False,None,43.528455,4.958107,43.528546,4.958099
1927731,"09/19/2011 Istres, Provence-Alpes-Côte d'Azur,...",3509.1,1231,1231,15.0,Run,Run,None,2011-09-19T04:35:45Z,2011-09-19T06:35:45Z,...,2294408,2011-09-19T04:35:45Z,0,0,False,None,43.528494,4.958120,43.528524,4.958041
1927730,"09/16/2011 Istres, Provence-Alpes-Côte d'Azur,...",3739.8,1387,1387,0.0,Run,Run,None,2011-09-16T04:39:17Z,2011-09-16T06:39:17Z,...,2294407,2011-09-16T04:39:17Z,0,0,False,None,43.528485,4.958134,43.528540,4.958092
